# Logical shadow tomography: Convergence in number of samples

## Setup

In [ ]:
from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import numpy as np

from base.stabilizer import random_clifford_state, stabilizer_state, stoc_depolarize_map
from base.utils import stabilizer_projection_full

In [ ]:
plt.rcParams.update({"font.family": "serif", "font.size": 16})

## Experiment

In [ ]:
"""Set experimental parameters."""
nmin: int = 5  # Minimum number of physical qubits to use in code.
nmax: int = 15  # Maximum number of physical qubits to use in code.
step: int = 5  # Step between qubit values.

max_num_trajectories: int = 50_000  # Maximum number of trajectories to sample from.

p: float = 0.01  # Depolarizing noise rate.

njobs: int = 2  # Number of parallel workers to use when sampling trajectories.

In [ ]:
nvalues = np.arange(nmin, nmax + step, step)

In [ ]:
# Codes from http://www.codetables.de/.
# TODO: Generate random codes for any n instead of hardcoding.
codes = {
    5: ["ZZZZZ", "XZZXI", "IXZZX", "XIXZZ", "ZXIXZ"],
    10: [
        "XZIZIXIZZI",
        "-IYIZZYIIZZ",
        "-IIYZZYIXII",
        "-IZZYIYZIIZ",
        "-IZIIXYZYII",
        "IZZIZZXXZZ",
        "IIZZZZZIXZ",
        "IZZZZIIZZX",
        "ZZZZZZIIII",
    ],
    15: [
        "XZIZIXIZZIIIIII",
        "-IYIZZYIIZZIIIII",
        "IZXIZXIIIZZIIII",
        "-IZZYIYZIIZIIIII",
        "IIZZXXZZIZZIIII",
        "-IIZZIIYZZIYIIII",
        "-IZIZZZZYIZYIIII",
        "IIIZIZIZXZXIIII",
        "IZIZIIZIZXXIIII",
        "IIIIIIIIIIIXIII",
        "IIIIIIIIIIIIXII",
        "IIIIIIIIIIIIIXI",
        "IIIIIIIIIIIIIIX",
        "ZZZZZZIIIIIIIII",
    ],
}

In [ ]:
def sample_trajectory(p: float):
    # Initial stabilizer state.
    state = stabilizer_state(*stabilizers)
    gs0 = state.gs.copy()
    ps0 = state.ps.copy()

    # Apply single-qubit depolarizing noise.
    state = stoc_depolarize_map(state, p)

    # Do shadow tomography.
    obs = random_clifford_state(N)
    state.measure(obs)

    gs, ps, _, tmp_PsigmaP = stabilizer_projection_full(
        state.gs, state.ps, gs0[: N - 1].copy(), ps0[: N - 1].copy(), 0
    )
    _, _, _, tmp = stabilizer_projection_full(gs, ps, gs0[:N], ps0[:N], 0)

    return tmp_PsigmaP * tmp, tmp_PsigmaP

In [ ]:
"""Run the experiment."""
all_n_values = []
all_fidelities = []
for N in nvalues:
    print(f"Status: On n = {N}", end="\r")

    try:
        stabilizers = codes.get(N)
    except KeyError:
        raise ValueError(
            f"Unsupported value for `N`. I only know codes for these N values: {list(codes.keys())}."
        )

    all_values = Parallel(n_jobs=njobs)(
        delayed(sample_trajectory)(p) for _ in range(max_num_trajectories)
    )
    all_n_values.append(all_values)

In [ ]:
nsamples_values = np.linspace(1_000, max_num_trajectories, 100, dtype=int)

for (n, all_values) in zip(nvalues, all_n_values):
    fidelities = []
    for nsamples in nsamples_values:
        num, denom = np.sum(all_values[:nsamples], axis=0)
        fid = ((2 ** n + 1) * (num / nsamples) - 1) / (
            (denom / nsamples) * (2 ** n + 1) - 2
        )
        fidelities.append(fid)

    all_fidelities.append(fidelities)

In [ ]:
plt.figure(figsize=(9, 5))

for N, fids in zip(nvalues, all_fidelities):
    plt.plot(nsamples_values, fids, "--.", label=f"$[[n, k, d]] = [[{N}, 1, d]]$")

plt.xlabel("Number of samples")
plt.ylabel("Fidelity")
plt.legend();